In [1]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


make beautifulsoup object of website

In [81]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import bs4
import requests

r  = requests.get(url)

data = r.text

soup = bs4.BeautifulSoup(data, 'lxml')

convert website contents to dataframe

In [82]:
import pandas as pd
my_table = soup.find('table',{'class':'wikitable sortable'})
entries = my_table.find_all('td')

cols = ['postcode', 'borough', 'neighborhood']
data_dict = {'postcode': [], 'borough': [], 'neighborhood': []}

it = iter(entries)

while True:
    for col in cols:
        entry = next(it, False)
        if not entry:
            break
        if entry.string is None:
            data_dict[col].append(entry.contents[0].attrs['title'])
        else:
            data_dict[col].append(entry.string)
    else:
        continue
    
    break

df = pd.DataFrame(data_dict)
df = df[cols]

print(df[:10])

  postcode           borough            neighborhood
0      M1A      Not assigned           Not assigned

1      M2A      Not assigned           Not assigned

2      M3A        North York               Parkwoods
3      M4A        North York        Victoria Village
4      M5A  Downtown Toronto  Harbourfront (Toronto)
5      M5A  Downtown Toronto             Regent Park
6      M6A        North York        Lawrence Heights
7      M6A        North York          Lawrence Manor
8      M7A      Queen's Park           Not assigned

9      M8A      Not assigned           Not assigned



take out the rows with a borough that is not assigned

In [47]:
df = df[df.borough != 'Not assigned']
print(df[:10])

   postcode           borough            neighborhood
2       M3A        North York               Parkwoods
3       M4A        North York        Victoria Village
4       M5A  Downtown Toronto  Harbourfront (Toronto)
5       M5A  Downtown Toronto             Regent Park
6       M6A        North York        Lawrence Heights
7       M6A        North York          Lawrence Manor
8       M7A      Queen's Park           Not assigned

10      M9A         Etobicoke        Islington Avenue
11      M1B       Scarborough          Rouge, Toronto
12      M1B       Scarborough        Malvern, Toronto


merge the neighborhoods that have the same postcode

In [79]:
def tmp(group_series):
    return pd.Series(data={'borough': group_series.iloc[0].borough, 'neighborhood': ', '.join([neighborhood.strip() for neighborhood in group_series.neighborhood])})
    
df = df.groupby('postcode').agg(tmp)

df.reset_index(inplace=True)

print(df[:20])

   postcode      borough                                       neighborhood
0       M1B  Scarborough                   Rouge, Toronto, Malvern, Toronto
1       M1C  Scarborough  Highland Creek (Toronto), Rouge Hill, Port Uni...
2       M1E  Scarborough  Guildwood, Morningside, Toronto, West Hill, To...
3       M1G  Scarborough                                    Woburn, Toronto
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough  East Birchmount Park, Ionview, Kennedy Park, T...
7       M1L  Scarborough  Clairlea, Golden Mile, Toronto, Oakridge, Toronto
8       M1M  Scarborough  Cliffcrest, Cliffside, Toronto, Scarborough Vi...
9       M1N  Scarborough                        Birch Cliff, Cliffside West
10      M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11      M1R  Scarborough                Maryvale, Toronto, Wexford, Toronto
12      M1S 

if the neighborhood is not assigned but the borough is, set the neighborhood to the borough

In [80]:
df.loc[df.neighborhood == 'Not assigned', 'neighborhood'] = df.loc[df.neighborhood == 'Not assigned', 'borough']

In [83]:
df.shape

(288, 3)